**TODO :**
- fix plotting: BUG when region in the second half of the alignment
- clarify the ticks and values: delete the first or not
- exception when region given vice verca, i.e. : (1000, 1)



In [1]:
from Bio import AlignIO
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
init_notebook_mode(connected=True)

import pandas as pd

In [2]:
"""simgen

This script allows the user to create similarity plots which are useful 
to analyze possible recombination events in DNA sequences. 

The simgen function accepts alignment file in fasta format

The output of simgen is plotly based similarity plot which is easy to manipulate.

This script requires that 'pandas', 'biopython' and 'plotly' by installed within the Python 
environment your are running this script in

"""

"simgen\n\nThis script allows the user to create similarity plots which are useful \nto analyze possible recombination events in DNA sequences. \n\nThe simgen function accepts alignment file in fasta format\n\nThe output of simgen is plotly based similarity plot which is easy to manipulate.\n\nThis script requires that 'pandas', 'biopython' and 'plotly' by installed within the Python \nenvironment your are running this script in\n\n"

In [3]:
#TODO: pdistance and draw_simplot should be private(don't forget to correct their calls inside simgen)

In [4]:
def _pdistance(seq1, seq2):
    """calculates pairwise distance between two sequences"""
    
    p = 0
    pairs = []
    
    for x in zip(seq1, seq2):
        if '-' not in x:
            pairs.append(x)
    
    for (x, y) in pairs:
        if x != y:
            p += 1
    length = len(pairs)
    
    try:
        dist = float(1 - p / length)  # '1 - p' to take plot 'upside down'
        return dist
    except ZeroDivisionError as e:
        print(e, ": perhaps your alignment contains only gaps")
    
    

In [5]:
def _draw_simplot(distance_data, tick_container):
    """draws similarity plot"""
    
    data = []    

    for key in distance_data.keys():
        trace = go.Scatter(y=distance_data[key], x=tick_container, name=key)
        data.append(trace)
    
    layout = go.Layout(
        title="similarity plot",
        xaxis=dict(
            title="nucleotide position"),
        yaxis=dict(
            title="sequence identity"),
        legend=dict(x=-0.1, y=1.5))
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
 
    

In [6]:
def _get_x_labels(left_border, right_border, shift):
    """creates tick labels"""
    
    tick_container = []      
    tick_container.append(left_border)
    
    while tick_container[-1] < right_border:
        tick_container.append(tick_container[-1] + shift)
        if tick_container[-1] > right_border:
            tick_container[-1] = right_border
            
    return tick_container
    

In [17]:
def _move_window(align, window, pot_rec, shift):
    """moves window"""
    distance_data = {}
    parents = list(range(0, len(align)))
    parents.remove(pot_rec)
    align_length = len(align[0, :])
    
    for par in parents:
        dist_container = []
        start = 0
        finish = shift

        while start < align_length:
            seq1 = align[pot_rec, start:finish].seq # here is a potential recombinant sequence slice
            seq2 = align[par, start:finish].seq  # here's a parent's slice
            dist_container.append(_pdistance(seq1, seq2)) #calculate pdistance, append to container
            start += shift
            finish = start + window

        distance_data[align[par].id] = dist_container
        
    return distance_data

In [23]:
def simgen(align, pot_rec, window=500, shift=100, region=False, draw=True):
    """slices the alignment, collects the distance data
    
    Parameters:
    -----------
    align: the location of the fasta alignment
    pot_rec: int
        the number of the sequence under study, starts with 0, 
        like the 'x' dimension in the numpy array 
    window: int
        sliding window size
    shift: int
        the step window slides downstream the alignment
    region: a tuple or a list of two integers
        the region of the alignment to analyze. the start and the end 
        nucleotide positions
    return_data: bool, optional
        return the data in pandas DataFrame 
    """
    assert window >=1, "wondow can't be a negative or zero"
    assert shift >= 1, "shift can't be a negative or zero" 
    
    if region:
        assert region[0] < region[1], "the value of the first nucleotide position should be less than the second one"
        align = AlignIO.read(align, "fasta")
        align = align[:, region[0] : region[1]]
        left_border = region[0]   # border for the first tick
        right_border = region[1]  # if region, 'right_border' is actual position
    else:
        align = AlignIO.read(align, "fasta")
        left_border = 1  # border for the first tick
        right_border = len(align[0, :])
    
    # creating tick labels for the plot
    ticks = _get_x_labels(left_border, right_border, shift)
    
    # calculating pairwise distance
    distance_data = _move_window(align, window, pot_rec, shift)
    
    
    if draw:
        _draw_simplot(distance_data, ticks) 
    else:
        data = pd.DataFrame(data=distance_data, index=ticks[1:]).T # [1:] to map data to index
        #data = distance_data # test line
        return data


In [19]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2)

In [16]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2)

In [36]:
data = simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(1,10000))

In [37]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(140000, 150000))

In [63]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(120000, 150000))

[120000, 120100, 120200, 120300, 120400, 120500, 120600, 120700, 120800, 120900, 121000, 121100, 121200, 121300, 121400, 121500, 121600, 121700, 121800, 121900, 122000, 122100, 122200, 122300, 122400, 122500, 122600, 122700, 122800, 122900, 123000, 123100, 123200, 123300, 123400, 123500, 123600, 123700, 123800, 123900, 124000, 124100, 124200, 124300, 124400, 124500, 124600, 124700, 124800, 124900, 125000, 125100, 125200, 125300, 125400, 125500, 125600, 125700, 125800, 125900, 126000, 126100, 126200, 126300, 126400, 126500, 126600, 126700, 126800, 126900, 127000, 127100, 127200, 127300, 127400, 127500, 127600, 127700, 127800, 127900, 128000, 128100, 128200, 128300, 128400, 128500, 128600, 128700, 128800, 128900, 129000, 129100, 129200, 129300, 129400, 129500, 129600, 129700, 129800, 129900, 130000, 130100, 130200, 130300, 130400, 130500, 130600, 130700, 130800, 130900, 131000, 131100, 131200, 131300, 131400, 131500, 131600, 131700, 131800, 131900, 132000, 132100, 132200, 132300, 132400,

** here's exception, if region given in the wrong way: first position is greater or equal than second :**

In [56]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(100000, 1))

AssertionError: the value of first nucleotide position should be less than the second one

In [61]:
simgen("../for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(555, 555))

AssertionError: the value of the first nucleotide position should be less than the second one

In [64]:
simgen("../for_recan_lsdv.fasta", window=500, shift=500, pot_rec=2)

[1, 501, 1001, 1501, 2001, 2501, 3001, 3501, 4001, 4501, 5001, 5501, 6001, 6501, 7001, 7501, 8001, 8501, 9001, 9501, 10001, 10501, 11001, 11501, 12001, 12501, 13001, 13501, 14001, 14501, 15001, 15501, 16001, 16501, 17001, 17501, 18001, 18501, 19001, 19501, 20001, 20501, 21001, 21501, 22001, 22501, 23001, 23501, 24001, 24501, 25001, 25501, 26001, 26501, 27001, 27501, 28001, 28501, 29001, 29501, 30001, 30501, 31001, 31501, 32001, 32501, 33001, 33501, 34001, 34501, 35001, 35501, 36001, 36501, 37001, 37501, 38001, 38501, 39001, 39501, 40001, 40501, 41001, 41501, 42001, 42501, 43001, 43501, 44001, 44501, 45001, 45501, 46001, 46501, 47001, 47501, 48001, 48501, 49001, 49501, 50001, 50501, 51001, 51501, 52001, 52501, 53001, 53501, 54001, 54501, 55001, 55501, 56001, 56501, 57001, 57501, 58001, 58501, 59001, 59501, 60001, 60501, 61001, 61501, 62001, 62501, 63001, 63501, 64001, 64501, 65001, 65501, 66001, 66501, 67001, 67501, 68001, 68501, 69001, 69501, 70001, 70501, 71001, 71501, 72001, 72501, 7

In [67]:
simgen("../for_recan_lsdv.fasta", window=500, shift=499, pot_rec=2)

[1, 500, 999, 1498, 1997, 2496, 2995, 3494, 3993, 4492, 4991, 5490, 5989, 6488, 6987, 7486, 7985, 8484, 8983, 9482, 9981, 10480, 10979, 11478, 11977, 12476, 12975, 13474, 13973, 14472, 14971, 15470, 15969, 16468, 16967, 17466, 17965, 18464, 18963, 19462, 19961, 20460, 20959, 21458, 21957, 22456, 22955, 23454, 23953, 24452, 24951, 25450, 25949, 26448, 26947, 27446, 27945, 28444, 28943, 29442, 29941, 30440, 30939, 31438, 31937, 32436, 32935, 33434, 33933, 34432, 34931, 35430, 35929, 36428, 36927, 37426, 37925, 38424, 38923, 39422, 39921, 40420, 40919, 41418, 41917, 42416, 42915, 43414, 43913, 44412, 44911, 45410, 45909, 46408, 46907, 47406, 47905, 48404, 48903, 49402, 49901, 50400, 50899, 51398, 51897, 52396, 52895, 53394, 53893, 54392, 54891, 55390, 55889, 56388, 56887, 57386, 57885, 58384, 58883, 59382, 59881, 60380, 60879, 61378, 61877, 62376, 62875, 63374, 63873, 64372, 64871, 65370, 65869, 66368, 66867, 67366, 67865, 68364, 68863, 69362, 69861, 70360, 70859, 71358, 71857, 72356, 728

In [69]:
simgen("../for_recan_lsdv.fasta", window=1, shift=1, pot_rec=2, region=(1,10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
10


In [24]:
simgen("../for_recan_lsdv.fasta", window=100, shift=100, pot_rec=2, region=(0,15500), draw=False)

,100,200,300,400,500,600,700,800,900,1000,...,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500
AF409138.1_Lumpy_skin_disease_virus_iso,0.99,1.0,1.0,1.0,0.98,1.0,1.0,0.98,0.99,1.0,...,0.98,1.0,0.99,1.0,0.98,0.97,0.98,1.0,1.0,0.99
mapping_consensus_on_AF409138.1_default,0.99,1.0,1.0,1.0,0.98,1.0,1.0,0.98,0.99,1.0,...,0.98,1.0,0.99,1.0,0.98,0.97,0.98,1.0,1.0,0.99
NODE_1_length_146178_cov_19.4988,0.99,1.0,1.0,1.0,0.98,1.0,1.0,0.98,0.99,1.0,...,0.98,1.0,0.99,1.0,0.98,0.97,0.98,1.0,1.0,0.99
contig_assembly_2,0.99,1.0,1.0,1.0,0.98,1.0,1.0,0.98,0.99,1.0,...,0.98,1.0,0.99,1.0,0.98,0.97,0.98,1.0,1.0,0.99


In [72]:
simgen("../for_recan_lsdv.fasta", window=1, shift=0, pot_rec=2, region=(1,10))

AssertionError: shift can't be a negative or zero

In [73]:
simgen("../for_recan_lsdv.fasta", window=1, shift=-1, pot_rec=2, region=(1,10))

AssertionError: shift can't be a negative or zero

In [89]:
simgen("../for_recan_lsdv.fasta", window=-1, shift=1, pot_rec=2, region=(1,10))

AssertionError: wondow can't be a negative or zero